In [5]:
print(1)

1


In [6]:
import psycopg2
from psycopg2.extras import RealDictCursor
import pandas as pd
from datetime import datetime
import json

In [7]:
DB_CONFIG = {
    'dbname': 'resource_tracker',  # replace with your db name
    'user': 'admin',            # replace with your username
    'password': 'Shakti!1997',   # replace with your password
    'host': 'localhost',
    'port': '5432'
}

In [8]:
def get_db_connection():
    """Create a database connection"""
    return psycopg2.connect(**DB_CONFIG)

In [9]:
conn = get_db_connection()

In [10]:
cur = conn.cursor()

In [11]:
cur.execute("SELECT * from servers;")

In [15]:
!pip install openstacksdk python-chi

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 10.6 MB/s eta 0:00:00m eta 0:00:010:0101
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.2/58.2 KB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.5/108.5 KB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.7/58.7 KB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.6/341.6 KB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 KB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 KB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.2/256.2 KB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.0/241.0 KB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.0/155.0 KB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.7/295.7 KB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━

In [16]:
import openstack, chi, chi.ssh, chi.network, chi.server, os

In [20]:
chi.use_site("KVM@TACC")
PROJECT_NAME = "CHI-231138"
chi.set("project_name", PROJECT_NAME)
from openstack import connection
# configure openstacksdk for actions unsupported by python-chi
os_conn = connection.Connection(
    region_name='KVM@TACC',
    auth={
        'auth_url': 'https://kvm.tacc.chameleoncloud.org:5000',
        'application_credential_id': '47f91411bfed466b8f71a58158d8be42',
        'application_credential_secret' : 'EMrkZVxMdcSmhkS-Slo_kVz45MtRfHAVcs2XPowp63cH1axciuhmNCxdv8HzlnlCnJy0LKKqGAIKwEYpFFBYYQ',
        'project_id': '13a1ac1ce275484caedc3394339486a1',
        'user_domain_id': '4c2da20c54a8081ff9ac30cd276553749e8f39b7f45e80cd53b9769c8e0cbb1f',
    },
    compute_api_version='2',
    identity_api_version= 3,
    auth_type= 'v3applicationcredential'
)

Now using KVM@TACC:
URL: https://kvm.tacc.chameleoncloud.org
Location: Austin, Texas, USA
Support contact: help@chameleoncloud.org


In [21]:
os_conn.list_servers()

Unauthorized: Error authenticating with application credential: Application credentials cannot request a scope. (HTTP 401) (Request-ID: req-73134428-6c38-454d-9998-25ea2a2ace87)